In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("hw4.ipynb")

# Homework 4: Text Analysis Using Twitter

**Due Date: Thursday, February 18th, 11:59 PM PST**

Welcome to the fourth homework assignment of Data 100/200! In this assignment, we will be exploring tweets from several high profile Twitter users.  

**Collaboration Policy**

Data science is a collaborative activity. While you may talk with others about the homework, we ask that you **write your solutions individually**. If you do discuss the assignments with others please **include their names** below.

**Collaborators**: *list collaborators here*

In [116]:
# Run this cell to set up your notebook
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

from ds100_utils import *

# Ensure that Pandas shows at least 280 characters in columns, so we can see full tweets
pd.set_option('max_colwidth', 280)
plt.style.use('fivethirtyeight')
sns.set()
sns.set_context("talk")

def horiz_concat_df(dict_of_df, head=None):
    """
    Horizontally concatenante multiple dataframes for easier visualization. 
    Each dataframe must have the same columns.
    """
    df = pd.concat([df.reset_index(drop=True) for df in dict_of_df.values()], axis=1, keys=dict_of_df.keys())
    if head is None:
        return df
    return df.head(head)

### Score Breakdown

Question | Points
--- | ---
1a | 1
1b | 1
1c | 3
2a | 2
2b | 2
2c | 2
2d | 2
2e | 2
2f | 1
3a | 1
3b | 1
3c | 1
4a | 1
4bi | 1
4bii | 1
4biii | 1
4biv | 2
4c | 1
4d | 2
4e | 2
4f | 2
5a | 2
5b | 2
**Total** | **36**

<br/><br/><br/>
<br/><br/><br/>
<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

## Question 1: Importing the Data


The data for this assignment was obtained using the [Twitter APIs](https://developer.twitter.com/en/docs/twitter-api).  To ensure that everyone has the same data and to eliminate the need for every student to apply for a Twitter developer account, we have collected a sample of tweets from several high-profile public figures.  The data is stored in the folder `data`.  Run the following cell to list the contents of the directory:

In [117]:
from os import listdir
for f in listdir("data"):
    print(f)

### Question 1a

Let's examine the contents of one of these files.  Using the [`open` function](https://docs.python.org/3/library/functions.html#open) and [`read` operation](https://docs.python.org/3/tutorial/inputoutput.html#methods-of-file-objects) on a python file object, read the first 1000 characters in `data/BernieSanders_recent_tweets.txt` and store your result in the variable `q1a`.  Then display the result so you can read it.

**Caution:** Viewing the contents of large files in a Jupyter notebook could crash your browser.  Be careful not to print the entire contents of the file.

**Hint:** You might want to try to use `with`:

```python
with open("filename", "r") as f:
    f.read(2)
```


<!--
BEGIN QUESTION
name: q1a
points: 1
-->

In [118]:
q1a = ...

In [ ]:
grader.check("q1a")

### Question 1b

What format is the data in? Answer this question by entering the letter corresponding to the right format in the variable `q1b` below.

<ol style="list-style-type: lower-alpha;">
    <li>CSV</li>
    <li>HTML</li>
    <li>JavaScript Object Notation (JSON)</li>
    <li>Excel XML</li>
</ol>

<!--
BEGIN QUESTION
name: q1b
points: 
    - 0
    - 1
-->

In [123]:
q1b = "..."

In [ ]:
grader.check("q1b")

### Question 1c

Pandas has built-in readers for many different file formats including the file format used here to store tweets.  To learn more about these, check out the documentation for [`pd.read_csv`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html), [`pd.read_html`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_html.html), [`pd.read_json`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_json.html), and [`pd.read_excel`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_excel.html).  

1. Use one of these functions to populate the `tweets` dictionary with the tweets for: `AOC`, `Cristiano`, and `elonmusk`. The keys of `tweets` should be the handles of the users, which we have provided in the cell below, and the values should be the dataframes.
2. Set the index of each dataframe to correspond to the `id` of each tweet.  



**Hint:** You might want to first try loading one of the DataFrames before trying to complete the entire question.

<!--
BEGIN QUESTION
name: q1c
points: 3
-->

In [126]:
tweets = {
    "AOC": ...,
    "Cristiano": ...,
    "elonmusk": ...,
}

In [ ]:
grader.check("q1c")

If you did everything correctly, the following cells will show you the first 5 tweets for Elon Musk (and a lot of information about those tweets).

In [135]:
tweets["elonmusk"].head()

<br/><br/><br/>
<br/><br/><br/>
<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />


## Question 2:  Source Analysis


In some cases, the Twitter feed of a public figure may be partially managed by a public relations firm. In these cases, the device used to post the tweet may help reveal whether it was the individual (e.g., from an iPhone) or a public relations firm (e.g., TweetDeck).  The tweets we have collected contain the source information but it is formatted strangely :(

In [136]:
tweets["Cristiano"][["source"]]

In this question we will use a regular expression to convert this messy HTML snippet into something more readable.  For example: `<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>` should be `Twitter for iPhone`. 


### Question 2a

We will first use the Python `re` library to cleanup the above test string.  In the cell below, write a regular expression that will match the **HTML tag** and assign it to the variable `q2a_pattern`. We then use the `re.sub` function to substitute anything that matches the pattern with an empty string `""`.

An HTML tag is defined as a `<` character followed by zero or more non-`>` characters, followed by a `>` character. That is `<a>` and `</a>` are both considered _separate_ HTML tags.

<!--
BEGIN QUESTION
name: q2a
points: 2
-->

In [137]:
q2a_pattern = r"..."
test_str = '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>'
re.sub(q2a_pattern, "", test_str)

In [ ]:
grader.check("q2a")

### Question 2b

Rather than writing a regular expression to detect and remove the HTML tags we could instead write a regular expression to **capture** the device name between the angle brackets.  Here we will use [**capturing groups**](https://docs.python.org/3/howto/regex.html#grouping) by placing parenthesis around the part of the regular expression we want to return.  For example, to capture the `21` in the string `08/21/83` we could use the pattern `r"08/(..)/83"`.  


**Hint:** The output of the following cell should be `['Twitter for iPhone']`.

<!--
BEGIN QUESTION
name: q2b
points: 2
-->

In [143]:
q2b_pattern = r"..."
test_str = '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>'
re.findall(q2b_pattern, test_str)

In [ ]:
grader.check("q2b")

### Question 2c

Using either of the two regular expressions you just created and [`Series.str.replace`](https://pandas.pydata.org/docs/reference/api/pandas.Series.replace.html) or [`Series.str.extract`](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.extract.html), add a new column called `"device"` to **all** of the dataframes in `tweets` containing just the text describing the device (without the HTML tags).

<!--
BEGIN QUESTION
name: q2c
points: 2
-->

In [148]:
tweets = ...

In [ ]:
grader.check("q2c")

### Question 2d

To examine the most frequently used devices by each individual, implement the `most_freq` function that takes in a `Series` and returns a new `Series` containing the `k` most commonly occuring entries in the first series, where the values are the counts of the entries and the indices are the entries themselves.

For example: 
```python
most_freq(pd.Series(["A", "B", "A", "C", "B", "A"]), k=2)
```
would return:
```
A    3
B    2
dtype: int64
```




**Hint** Consider using `value_counts`, `sort_values`, `head`, and/or `nlargest`.  What might be the most efficient implementation?

<!--
BEGIN QUESTION
name: q2d
points: 2
-->

In [157]:
def most_freq(series, k = 5):
    ...

most_freq(tweets["Cristiano"]['device'])

In [ ]:
grader.check("q2d")

### Question 2e

Run the following two cells to compute a table and plot describing the top 5 most commonly used devices for each user. 

In [162]:
device_counts = pd.DataFrame(
    [most_freq(tweets[name]['device']).rename(name)
     for name in tweets]
).fillna(0)
device_counts

In [163]:
plt.figure(figsize=[15,6])
sns.barplot(x="index", y="value", hue="variable", data=device_counts.T.reset_index().melt(id_vars="index"))
plt.xticks(rotation=45)
plt.title("Count of Tweets by Source")
plt.ylabel("Count")
plt.xlabel("Source")
plt.legend(title="Handle");

<!-- BEGIN QUESTION -->

What might we want to investigate further?  Write a few sentences below and be prepared to discuss in next week's small group meeting.

<!--
BEGIN QUESTION
name: q2e
points: 2
manual: true
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### Question 2f

We just looked at the top 5 most commonly used devices for each user. However, we used the number of tweets as a measure, when it might be better to compare these distributions by comparing _proportions_ of tweets. Why might proportions of tweets be better measures than numbers of tweets?

<!--
BEGIN QUESTION
name: q2f
points: 1
manual: true

-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



<br/><br/><br/>
<br/><br/><br/>
<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />


## Question 3: When?

Now that we've explored the sources of each of the tweets, we will perform some time series analysis. A look into the temporal aspect of the data could reveal insights about how a user spends their day, when they eat and sleep, etc. In this question, we will focus on the time at which each tweet was posted.


### Question 3a

Complete the following function `add_hour` that adds a new column `hour` to a tweets `DataFrame` based on the column `time_col` containing the timestamps. The `hour` column should contain the hour of the day as floating point number computed by:

$$
\text{hour} + \frac{\text{minute}}{60} + \frac{\text{second}}{60^{2}}
$$

**Hint:** See the following link for an example of working with timestamps using the [`dt` accessors](https://pandas.pydata.org/pandas-docs/stable/user_guide/basics.html#dt-accessor). You should use the `created_at` column to calculate the hour.

<!--
BEGIN QUESTION
name: q3a
points: 1
-->

In [164]:
def add_hour(df, time_col="created_at", result_col="hour"):
    ...
    return df

tweets = {handle: add_hour(df) for handle, df in tweets.items()}
tweets["AOC"]["hour"].head()

In [ ]:
grader.check("q3a")

### Question 3b

With our new `hour` column, let's take a look at the distribution of tweets for each user by time of day. The following cell helps create a density plot on the number of tweets based on the hour they are posted. 

The function `bin_df` takes in a dataframe, an array of bins, and a column name and bins the the values in the specified column, returning a dataframe with the bin lower bound and the number of elements in the bin. This function uses [`pd.cut`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.cut.html), a pandas utility for binning numerical values that you may find helpful in the future.

Run the cell and answer the following question about the plot.

In [170]:
def bin_df(df, bins, colname):
    binned = pd.cut(df[colname], bins).value_counts().sort_index()
    return pd.DataFrame({"counts": binned, "bin": bins[:-1]})

hour_bins = np.arange(0, 24.5, .5)
binned_hours = {handle: bin_df(df, hour_bins, "hour") for handle, df in tweets.items()}

make_line_plot(binned_hours, "bin", "counts", title="Distribution of Tweets by Time of Day",
               xlabel="Hour", ylabel="Number of Tweets")

<!-- BEGIN QUESTION -->

Compare Cristiano's distribution with those of AOC and Elon Musk. In particular, compare the distributions before and after hour 6. What differences did you notice? What might be a possible cause of that? Do the data plotted above seem reasonable?

<!--
BEGIN QUESTION
name: q3b
points: 1
manual: true
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->

### Question 3c

To account for different locations of each user in our analysis, we will next adjust the `created_at` timestamp for each tweet to the respective timezone of each user. Complete the following function `convert_timezone` that takes in a tweets `DataFrame` and a dictionary containing each user's timezone `new_tz` and add a new column `converted_time` that has the adjusted `created_at` timestamp for each tweet. The timezone for each user is provided in `timezones`.

**Hint:** Again, please see the following link for an example of working with [`dt` accessors](https://pandas.pydata.org/pandas-docs/stable/user_guide/basics.html#dt-accessor).

<!--
BEGIN QUESTION
name: q3c
points: 1
-->

In [171]:
def convert_timezone(df, new_tz):
    ...
    return df

timezones = {"AOC": "EST", "Cristiano": "Europe/Lisbon", "elonmusk": "America/Los_Angeles"}

tweets = {handle: convert_timezone(df, tz) for (handle, df), tz in zip(tweets.items(), timezones.values())}

In [ ]:
grader.check("q3c")

With our adjusted timestamps for each user based on their timezone, let's take a look again at the distribution of tweets by time of day.

In [176]:
tweets = {handle: add_hour(df, "converted_time", "converted_hour") for handle, df in tweets.items()}
binned_hours = {handle: bin_df(df, hour_bins, "converted_hour") for handle, df in tweets.items()}

make_line_plot(binned_hours, "bin", "counts", title="Distribution of Tweets by Time of Day (timezone-corrected)",
               xlabel="Hour", ylabel="Number of Tweets")

<br/><br/><br/>
<br/><br/><br/>
<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />


## Question 4: Sentiment


In the past few questions, we have explored the sources of the tweets and when they are posted. Although on their own, they might not seem particularly intricate, combined with the power of regular expressions, they could actually help us infer a lot about the users. In this section, we will continue building on our past analysis and specifically look at the sentiment of each tweet -- this would lead us to a much more direct and detailed understanding of how the users view certain subjects and people. 


---

How do we actually measure the sentiment of each tweet? In our case, we can use the words in the text of a tweet for our calculation! For example, the word "love" within the sentence "I love America!" has a positive sentiment, whereas the word "hate" within the sentence "I hate taxes!" has a negative sentiment. In addition, some words have stronger positive / negative sentiment than others: "I love America." is more positive than "I like America."

We will use the [VADER (Valence Aware Dictionary and sEntiment Reasoner)](https://github.com/cjhutto/vaderSentiment) lexicon to analyze the sentiment of AOC's tweets. VADER is a lexicon and rule-based sentiment analysis tool that is specifically attuned to sentiments expressed in social media which is great for our usage.

The VADER lexicon gives the sentiment of individual words. Run the following cell to show the first few rows of the lexicon:

In [177]:
print(''.join(open("vader_lexicon.txt").readlines()[:10]))

As you can see, the lexicon contains emojis too! Each row contains a word and the *polarity* of that word, measuring how positive or negative the word is.

### VADER Sentiment Analysis

The creators of VADER describe the tool’s assessment of polarity, or “compound score,” in the following way:

“The compound score is computed by summing the valence scores of each word in the lexicon, adjusted according to the rules, and then normalized to be between -1 (most extreme negative) and +1 (most extreme positive). This is the most useful metric if you want a single unidimensional measure of sentiment for a given sentence. Calling it a 'normalized, weighted composite score' is accurate.”

As you can see, VADER doesn't "read" sentences, but works by parsing sentences into words, assigning a preset generalized score from their testing sets to each word separately. 

VADER relies on humans to stabilize its scoring. The creators use Amazon Mechanical Turk, a crowdsourcing survey platform, to train its model. Its training data consists of a small corpus of tweets, New York Times editorials and news articles, Rotten Tomatoes reviews, and Amazon product reviews, tokenized using the natural language toolkit (NLTK). Each word in each dataset was reviewed and rated by at least 20 trained individuals who had signed up to work on these tasks through Mechanical Turk. 

### Question 4a

Let's first load in the data containing all the sentiments. Read `vader_lexicon.txt` into a DataFrame called `sent`. The index of the DataFrame should be the words in the lexicon. `sent` should have one column named `polarity`, storing the polarity of each word.

**Hint:** The `pd.read_csv` function may help here. Since the file is tab-separated, be sure to set `sep='\t'` in your call to `pd.read_csv`.

<!--
BEGIN QUESTION
name: q4a
points: 1
-->

In [178]:
sent = ...
sent.head()

In [ ]:
grader.check("q4a")

### Question 4b

Before further analysis, we will need some more tools that can help us extract the necessary information and clean our data.

Complete the following regular expressions that will help us match part of a tweet that we are either interested in learning more about or want to remove.

#### Part 1
Assign a regular expression to a new variable `punct_re` that captures all of the punctuations within a tweet. We consider punctuation to be any non-word, non-whitespace character.

**Note**: A word character is any character that is alphanumeric or an underscore. A whitespace character is any character that is a space, a tab, a new line, or a carriage return.

<!--
BEGIN QUESTION
name: q4bi
points: 1
-->

In [187]:
punct_re = r'...'

re.sub(punct_re, " ", tweets["AOC"].iloc[0]["full_text"])

In [ ]:
grader.check("q4bi")

#### Part 2
Assign a regular expression to a new variable `mentions_re` that matches any mention in a tweet. Your regular expression should use a capturing group to extract the user's username in a mention.

**Hint**: a user mention within a tweet always starts with the `@` symbol and is followed by a series of word characters (with no space in between). For more explanations on what a word character is, check out the **Note** section in Part 1.

<!--
BEGIN QUESTION
name: q4bii
points: 1
-->

In [192]:
mentions_re = r'...'

re.findall(mentions_re, tweets["AOC"].iloc[0]["full_text"])

In [ ]:
grader.check("q4bii")

#### Part 3

Assign a regular expression to a new variable `hashtag_re` that matches any of the hash tags within a tweet. Your regular expression should use a capturing group to extract the hashtag text (excluding the `#`).

**Hint**: A tweet has a hashtag if it has the character `#` anywhere in the tweet. In addition, the `#` character must be preceded by a whitespace character (spaces, tabs, etc.) or any charater that isn't underscore or alphanumeric. It also needs to be followed by at least one alphanumeric or underscore character.  

<!--
BEGIN QUESTION
name: q4biii
points: 1
-->

In [197]:
hashtag_re = r'...'

re.findall(hashtag_re, tweets["AOC"].loc[1337426463709073410]["full_text"])

In [ ]:
grader.check("q4biii")

#### Part 4

Assign a regular expression to a new variable `weblink_re` that captures all of the web links within a tweet.

**Hint**: A web link always starts with `http://` or `https://`. It is then followed by `www.`, the domain name and any additional subdomain names (subdomian names are separated by `/`). For example, for the web link `https://www.ds100.org/sp21/syllabus`, we are looking at the domain `ds100.org` with the subdomain `sp21` and its inner sudomain `syllabus`. The domain and subdomain names can be any number of consecutive alphanumeric characters.

**Note**: For simplicity, you can assume that in our case, the domain always starts with `www`. You may also assume that a web link will never end in `/`. For example, the weblink `https://www.ds100.org/` will not be considered valid for this question. It will be `https://www.ds100.org` instead.

<!--
BEGIN QUESTION
name: q4biv
points: 2
-->

In [202]:
weblink_re = r'...'

re.search(weblink_re, "a link to https://www.google.com!")

In [ ]:
grader.check("q4biv")

### Tweet Sentiments and User Mentions

As you have seen in the previous part of this question, there are actually a lot of interesting components that we can extract out of a tweet for further analysis! For the rest of this question though, we will focus on one particular case: the sentiment of each tweet in relation to the users mentioned within it. 

To calculate the sentiments for a sentence, we will follow this procedure:

1. Remove the punctuation from each tweet so we can analyze the words.
2. For each tweet, find the sentiment of each word.
3. Calculate the sentiment of each tweet by taking the sum of the sentiments of its words.

### Question 4c

Let's use our `punct_re` regular expression from the previous part to clean up the text a bit more! The goal here is to remove all of the punctuations to ensure words can be properly matched with those from VADER to actually calculate the full sentiment score.

Complete the following function `sanitize_texts` that takes in a table `df` and adds a new column `clean_text` by converting all characters in its original `full_text` column to lower case and replace all instances of punctuations with a space character.

<!--
BEGIN QUESTION
name: q4c
points: 1
-->

In [210]:
def sanitize_texts(df):
    df["clean_text"] = ...
    return df

tweets = {handle: sanitize_texts(df) for handle, df in tweets.items()}
tweets["AOC"]["clean_text"].head()

In [ ]:
grader.check("q4c")

### Question 4d
With the texts sanitized, we can now extract all the user mentions from tweets. 

Complete the following function `extract_mentions` that takes in the **`full_text` column** from a tweets `DataFrame` and uses `mentions_re` to extract all the mentions in a `pd.Series`. The returned series, renamed to `mentions`, is single-indexed and has all lower-cased characters with a separate row for each mention.

<!--
BEGIN QUESTION
name: q4d
points: 2
-->

In [215]:
def extract_mentions(full_texts):
    mentions = ...
    return mentions[["mentions"]]

mentions = {handle: extract_mentions(df["full_text"]) for handle, df in tweets.items()}
horiz_concat_df(mentions).head()

In [ ]:
grader.check("q4d")

### Tidying Up the Data

Now, let's convert the tweets into what's called a [*tidy format*](https://cran.r-project.org/web/packages/tidyr/vignettes/tidy-data.html) to make the sentiments easier to calculate. We will use the `clean_text` column of each dataframe to create a tidy table, which will be returned by `to_tidy_format`. The index of the table will be the IDs of the tweets, repeated once for every word in the tweet. It has two columns:

1. `num`: The location of the word in the tweet. For example, if the tweet was "i love america", then the location of the word "i" is 0, "love" is 1, and "america" is 2.
2. `word`: The individual words of each tweet.

Run the following cell to convert the table into the tidy format. Take a look at the first 5 rows from the "tidied" tweets dataframe for AOC and see if you can find out how the structure has changed.

**Note**: Although there is no work needed on your part, we have referenced a few more advanced pandas methods you might have not seen before -- you should definitely look them up in the documentation when you have a chance, as they are quite powerful in restructuring a dataframe and converting one into a useful intermediate state!

In [221]:
def to_tidy_format(df):
    tidy = (
        df["clean_text"]
        .str.split()
        .explode()
        .to_frame()
        .rename(columns={"clean_text": "word"})
    )
    return tidy

tidy_tweets = {handle: to_tidy_format(df) for handle, df in tweets.items()}
tidy_tweets["AOC"].head()

### Adding in the Polarity Score

Now that we have this table in the tidy format, it becomes much easier to find the sentiment of each tweet: we can join the table with the lexicon table. 

The following `add_polarity` function adds a new `polarity` column to the `df` table. The `polarity` column contains the sum of the sentiment polarity of each word in the text of the tweet.

**Note**: Again, though there is no work needed on your part, it is important for you to go through how we set up this method and actually understand what each method is doing.

In [222]:
def add_polarity(df, tidy_df):
    df["polarity"] = (
        tidy_df
        .merge(sent, how='left', left_on='word', right_index=True)
        .reset_index()
        .loc[:, ['id', 'polarity']]
        .groupby('id')
        .sum()
        .fillna(0)
    )
    return df

tweets = {handle: add_polarity(df, tidy_df) for (handle, df), tidy_df in \
          zip(tweets.items(), tidy_tweets.values())}
tweets["AOC"][["clean_text", "polarity"]].head()

### Question 4e
Finally, with our polarity column in place, we can finally explore how the sentiment of each tweet relates to the user(s) mentioned in it. 

Complete the following function `mention_polarity` that takes in a mentions dataframe `mentions` and the original tweets dataframe `df` and returns a series where the mentioned users are the index and the corresponding mean sentiment scores of the tweets mentioning them are the values.

**Hint**: You should consider joining tables together in this question.

<!--
BEGIN QUESTION
name: q4e
points: 2
-->

In [223]:
def mention_polarity(df, mention_df):

aoc_mention_polarity = mention_polarity(tweets["AOC"],mentions["AOC"]).sort_values(ascending=False)
aoc_mention_polarity

In [ ]:
grader.check("q4e")

<!-- BEGIN QUESTION -->

### Question 4f

When grouping by mentions and aggregating the polarity of the tweets, what aggregation function should we use? What might be some drawbacks of using the mean?

<!--
BEGIN QUESTION
name: q4f
points: 2
manual: true
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



<br/><br/><br/>
<br/><br/><br/>
<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />


## Question 5: You Do EDA

Congratulations! You have finished all of the preliminary analysis on AOC, Cristiano, and Elon Musk's recent tweets. 

As you might have recognized, there is still far more to explore within the data and build upon what we have uncovered so far. In this open-ended question, we want you to come up with a new perspective that can expand upon our analysis of the sentiment of each tweet. 

For this question, you will perform some text analysis on our `tweets` dataset. Your analaysis should have two parts:

1. a piece of code that manipulates `tweets` in some way and produces informative output (e.g. a dataframe, series, or plot)
2. a short (4-5 sentence) description of the findings of your analysis: what were you looking for? What did you find? How did you go about answering your question?

Your work should involve text analysis in some way, whether that's using regular expressions or some other form.

To aid you in creating plots, we provide the plotting helper functions in the table below. These are same helpers we have used throughout this notebook, and all accept dictionaries with a similar structure to `tweets`. That being said, if you know how to make plots, please do so! You'll be learning how to use the libraries that we're using in the helpers starting next week. 

| Helper | Description |
|--------|-------------|
| `make_bar_plot` | Plot side-by-side bar plots of data like [`plt.bar`](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.bar.html) |
| `make_histogram` | Plot overlaid histograms of data like [`plt.hist`](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.hist.html) |
| `make_line_plot` | Plot overlaid line plots of data like [`plt.plot`](https://matplotlib.org/3.3.3/api/_as_gen/matplotlib.pyplot.plot.html) |
| `make_scatter_plot` | Plot overlaid scatter plots of data like [`plt.scatter`](https://matplotlib.org/3.3.3/api/_as_gen/matplotlib.pyplot.scatter.html) |

Each of the provided helpers is in `ds100_utils.py` and has a comprehensive docstring. You can read the docstring by calling `help` on the plotting function:

In [229]:
help(make_line_plot)

To assist you in getting started, here are a few ideas for this you can analyze for this question:

- dig deeper into when devices were used
- how sentiment varies with time of tweet
- use other regexes from 4b to perform additional analysis
- examine sentiment of tweets over time

In general, try to combine the analyses from earlier questions or create new analysis based on the scaffolding we have provided.

This question is worth 4 points and will be graded based on this rubric:

| | 2 points | 1 point | 0 points |
|-----|-----|-----|-----|
| **Code** | Produces a mostly informative plot or pandas output that addresses the question posed in the student's description and uses at least one of the following pandas dataframe/series methods: `groupby`, `agg`, `merge`, `pivot_table`, `str`, `apply` | Attempts to produce a plot or manipulate data but the output is unrelated to the proposed question, or doesn't utilize at least one of the listed methods | No attempt at writing code |
| **Description** | Describes the analysis question and procedure comprehensively and summarizes results correctly | Attempts to describe analysis and results but description of results is incorrect or analysis of results is disconnected from the student’s original question | No attempt at writing a description |

**You will be discussing this question and presenting your notebook in your small group meetings next week.**

<!-- BEGIN QUESTION -->

### Question 5a

Use this space to put your EDA code.

<!--
BEGIN QUESTION
name: q5a
manual: true
points: 2
-->

In [230]:
# perform your text analysis here

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### Question 5b

Use this space to pur your EDA description.

<!--
BEGIN QUESTION
name: q5b
manual: true
points: 2
-->

_Write your description here._

<!-- END QUESTION -->



## Congratulations! You have finished Homework 4!

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export()